# [GZIP](http://www.gzip.org)

* `gzip` (GNU zip) is a open-source text compressor, written by Mark Adler and Jean-loup Gailly.

* It is described in [RFC 1951](https://www.ietf.org/rfc/rfc1951.txt) and [RFC 1952](https://www.ietf.org/rfc/rfc1952.txt).

### DEFLATE algorithm

1. Divide the input into blocks.
2. LZ77 encode each block.
3. Compute a Huffman tree for the offsets.
4. Huffman encode the Huffman tree using a static probabilistic model.
3. Huffman encode each block using an adaptive probabilistic model.


| lena                                                     | peppers                                                             |
|----------------------------------------------------------|--------------------------------------------------------------------|
| <img src="01-gzip/lena-gray.png" style="width: 400px;"/> | <img src="01-gzip/peppers-gray.png" style="width: 400px;"/> |

| boats                                                    | zelda                                                |
|----------------------------------------------------------|------------------------------------------------------|
| <img src="01-gzip/boats.png" style="width: 400px;"/>     | <img src="01-gzip/zelda.png" style="width: 400px;"/> |